In [ ]:
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from unsloth import FastVisionModel
import torch
from datasets import load_dataset
from transformers import TextStreamer
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# 1. Load the model

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)

model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules      = True,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

==((====))==  Unsloth 2025.7.3: Fast Mllama patching. Transformers: 4.53.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

Unsloth: Making `model.base_model.model.model.vision_model.transformer` require gradients


In [ ]:
# 2. Load the dataset

dataset = load_dataset("unsloth/Radiology_mini", split = "train")
instruction = "You are an expert radiographer. Describe accurately what you see in this image."

def convert_to_conversation(sample):
    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : sample["image"]} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : sample["caption"]} ]
        },
    ]
    return { "messages" : conversation }
pass

converted_dataset = [convert_to_conversation(sample) for sample in dataset]

README.md:   0%|          | 0.00/610 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/481M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/79.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1978 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/327 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt
from PIL import Image
import os

# Try to load the dataset - replace this with the actual dataset name if known
try:
    # First try the standard dataset loading
    dataset = load_dataset("unsloth/Radiology_mini", split = "train")
    print(f"Loaded dataset: {dataset}")
except Exception as e:
    print(f"Standard dataset loading failed: {e}")

# If we successfully loaded the dataset, display the first image
if 'dataset' in locals():
    try:
        # Try to access the image
        image = dataset[0]["image"]
        print("Successfully accessed image from dataset[0]")

        # Create a figure with two subplots side by side
        fig, axes = plt.subplots(1, 2, figsize=(20, 10))

        # Display grayscale image on the left
        axes[0].imshow(image, cmap='gray')
        axes[0].set_title("Dataset[0] Image (Grayscale)")
        axes[0].axis('off')

        # Display original colormap image on the right
        axes[1].imshow(image)
        axes[1].set_title("Dataset[0] Image (Original Colormap)")
        axes[1].axis('off')

        # Adjust layout and show the figure
        plt.tight_layout()
        plt.show()

        # Show dataset caption if available
        if "caption" in dataset[0]:
            print(f"Image caption: {dataset[0]['caption']}")

    except Exception as e:
        print(f"Error accessing or displaying image: {e}")


In [ ]:
# 3. Before training

FastVisionModel.for_inference(model)
image = dataset[0]["image"]
instruction = "You are an expert radiographer. Describe accurately what you see in this image."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

print("\nBefore training:\n")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)


Before training:

This radiograph appears to be a panoramic view of the upper jaw. The key features are:
- The bones that make up the floor and the lateral walls of the left maxillary sinus have been displaced. These have displaced a significant volume of air, with the formation of a cavity that would usually contain air or gas, into the maxillary sinus.
- The bone between the left upper teeth, near the central incisors, shows signs of a previous tooth extraction.
- The tooth next to it on the right side of the mouth, which is the first left bicuspid or premolar, shows signs of infection in its surrounding bone.



In [ ]:
# 4. Training

FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer),
    train_dataset = converted_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        max_seq_length = 2048,
    ),
)

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

trainer_stats = trainer.train()

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
10.279 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,978 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 67,174,400 of 10,737,395,235 (0.63% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.564400
2,3.632900
3,3.377200
4,3.180400
5,2.751900
6,2.532800
7,2.164800
8,1.977400
9,1.674800
10,1.312600


Step,Training Loss
1,3.564400
2,3.632900
3,3.377200
4,3.180400
5,2.751900
6,2.532800
7,2.164800
8,1.977400
9,1.674800
10,1.312600


1307.8885 seconds used for training.
21.8 minutes used for training.
Peak reserved memory = 10.568 GB.
Peak reserved memory for training = 0.289 GB.
Peak reserved memory % of max memory = 26.716 %.
Peak reserved memory for training % of max memory = 0.731 %.


In [ ]:
# 5. After training

print("\nAfter training:\n")
FastVisionModel.for_inference(model)
image = dataset[0]["image"]
instruction = "You are an expert radiographer. Describe accurately what you see in this image."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1, min_p = 0.1)


After training:

Panoramic radiograph of the patient showing the bilateral mandibular tori (arrows) in the area of the anterior teeth and the maxillary tori (arrows) in the area of the upper molars.<|eot_id|>


In [ ]:
# 6. Save the model

model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

[]

In [ ]:
!huggingface-cli login 


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `llama-3.1-demo` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authen

In [ ]:
model.save_pretrained_merged("devashish07/Multiodel-Radiology-Finetuing-LLAMA-3.2-11B", tokenizer,)
model.push_to_hub_merged("devashish07/Multiodel-Radiology-Finetuing-LLAMA-3.2-11B", tokenizer)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00005.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  20%|██        | 1/5 [00:36<02:24, 36.02s/it]

...

Unsloth: Merging weights into 16bit:  80%|████████  | 4/5 [06:26<01:36, 96.78s/it]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 5/5 [06:59<00:00, 83.81s/it]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q gradio unsloth

In [ ]:
# Test.py

import os
from unsloth import FastVisionModel
import torch
from transformers import TextStreamer
from PIL import Image

def main():
    # Load the LoRA model from the local directory
    model_path = "devashish07/Multiodel-Radiology-Finetuing-LLAMA-3.2-11B"

    # Load the model and tokenizer
    print("Loading model and tokenizer...")

    # Explicitly set device_map to 'cuda' if available, otherwise 'cpu'
    device_map = "cuda" if torch.cuda.is_available() else "cpu"

    model, tokenizer = FastVisionModel.from_pretrained(
        model_path,  # Base model
       # adapter_name=model_path,                  # LoRA adapter
        load_in_4bit=True,
        use_gradient_checkpointing="unsloth",
        device_map=device_map # Use the determined device
    )

    # Set model for inference
    FastVisionModel.for_inference(model)

    # Load your image (replace 'your_image.jpg' with the path to your image)
    image_path = "/content/image.jpg"  # Update this to your image path
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        print("Please update the image_path variable with your actual image path.")
        return

    image = Image.open(image_path)

    # Create the instruction for the model
    instruction = "You are an expert radiographer. Describe accurately what you see in this image."

    # Format the messages with the image and instruction
    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": instruction}
        ]}
    ]

    # Apply chat template and prepare inputs
    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

    # Get device
    print(f"Using device: {device_map}")

    # Tokenize inputs
    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to(device_map)

    # Generate response
    print("\nGenerating response:\n")
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    _ = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=128,
        use_cache=True,
        temperature=1,
        min_p=0.1
    )

if __name__ == "__main__":
    main()

==((====))==  Unsloth 2025.7.4: Fast Mllama patching. Transformers: 4.53.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c20f6cf5c9a5e2acc4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Gradio APP

In [ ]:
import os
import gradio as gr
from unsloth import FastVisionModel
import torch
from transformers import TextStreamer
from PIL import Image

# Load model and tokenizer once
model_path = "devashish07/Multiodel-Radiology-Finetuing-LLAMA-3.2-11B"
device_map = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[LOG] Loading model on device: {device_map}")

model, tokenizer = FastVisionModel.from_pretrained(
    model_path,
    adapter_name=model_path,
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth",
    device_map=device_map
)

FastVisionModel.for_inference(model)
print("[LOG] Model and tokenizer loaded.")

instruction = "You are an expert radiographer. Describe accurately what you see in this image."

def analyze_image(image):
    if image is None:
        return "No image selected!"
    try:
        messages = [
            {"role": "user", "content": [
                {"type": "image"},
                {"type": "text", "text": instruction}
            ]}
        ]
        input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
        inputs = tokenizer(
            image,
            input_text,
            add_special_tokens=False,
            return_tensors="pt",
        ).to(device_map)
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            use_cache=True,
            temperature=1,
            min_p=0.1
        )
        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        if instruction in output_text:
            output_text = output_text.split(instruction, 1)[1]

        prefixes_to_remove = [
            "You are an expert radiographer. Describe accurately what you see in this image.",
            "Describe accurately what you see in this image.",
            "assistant",
            "Assistant:",
            "I am an expert radiographer."
        ]

        for prefix in prefixes_to_remove:
            if output_text.startswith(prefix):
                output_text = output_text[len(prefix):]

        output_text = output_text.lstrip(":., \n")

        return output_text.strip()
    except Exception as e:
        return f"Error: {e}"


gr.HTML("""
<style>
.large-button button {
    font-size: 56px !important;
    font-weight: bold;
    padding: 12px 32px !important;  /* top-bottom, left-right */
    min-width: 200px !important;
}
</style>
""")



with gr.Blocks(css=".output-textbox textarea { font-size: 40px !important; }") as demo:
    gr.Markdown("<div style='font-size: 56px; font-weight: bold;'>Radiology Image Analyzer</div><div style='font-size: 30px;'>Upload an image to get an AI analysis.</div>")
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Image")
        with gr.Column():
            output = gr.Textbox(label="Model Output", lines=10, elem_classes="output-textbox")

    analyze_btn = gr.Button("Analyze", variant="primary", elem_classes="large-button")
    analyze_btn.click(analyze_image, inputs=image_input, outputs=output)

demo.launch(share=True)

[LOG] Loading model on device: cuda
==((====))==  Unsloth 2025.7.4: Fast Mllama patching. Transformers: 4.53.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

[LOG] Model and tokenizer loaded.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff8f1a68ee5a4454f6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
